In [ ]:
pip install optuna catboost xgboost lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.special import logit

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# 1️⃣ 데이터 로드
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv("./test.csv")

In [ ]:
# ID 제거
df_train.drop(columns=['ID'], inplace=True)
df_test_ids = df_test['ID']
df_test.drop(columns=['ID'], inplace=True)

In [ ]:
# 3️⃣ 타겟 변수 분리
X = df_train.drop(columns=['임신 성공 여부'], errors='ignore').copy()
y = df_train['임신 성공 여부']
df_test = df_test.drop(columns=['임신 성공 여부'], errors='ignore').copy()

In [ ]:
# 4️⃣ 유니크 값이 1개인 컬럼 삭제
drop_cols = ['불임 원인 - 여성 요인', '난자 채취 경과일']
X.drop(columns=drop_cols, inplace=True, errors='ignore')
df_test.drop(columns=drop_cols, inplace=True, errors='ignore')

# 5️⃣ 결측치 처리
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# 숫자형 변수 → 평균값으로 채우기
imputer_num = SimpleImputer(strategy="mean")
X[numeric_features] = imputer_num.fit_transform(X[numeric_features])
df_test[numeric_features] = imputer_num.transform(df_test[numeric_features])

# 범주형 변수 → 최빈값으로 채우기
imputer_cat = SimpleImputer(strategy="most_frequent")
X[categorical_features] = imputer_cat.fit_transform(X[categorical_features])
df_test[categorical_features] = imputer_cat.transform(df_test[categorical_features])

# 6️⃣ 범주형 변수 인코딩
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[categorical_features] = encoder.fit_transform(X[categorical_features])
df_test[categorical_features] = encoder.transform(df_test[categorical_features])

# 7️⃣ Feature Engineering (비율 Feature 추가)
# ✅ IVF 시술 비율 Feature 추가
X['IVF_비율'] = X.get('IVF 시술 횟수', 0) / (X.get('총 시술 횟수', 1) + 1e-6)
df_test['IVF_비율'] = df_test.get('IVF 시술 횟수', 0) / (df_test.get('총 시술 횟수', 1) + 1e-6)

# ✅ DI 시술 비율 Feature 추가
X['DI_비율'] = X.get('DI 시술 횟수', 0) / (X.get('총 시술 횟수', 1) + 1e-6)
df_test['DI_비율'] = df_test.get('DI 시술 횟수', 0) / (df_test.get('총 시술 횟수', 1) + 1e-6)


In [ ]:
# 🔟 Base Models 정의 (GradientBoosting 추가)
base_models = {
    "GradientBoosting": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "CatBoost": CatBoostClassifier(n_estimators=200, random_state=42, verbose=0),
    "XGBoost": XGBClassifier(n_estimators=200, random_state=42, use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1)
}

# ✅ K-Fold 설정
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ✅ Base Model 예측값 저장 공간
base_model_preds = np.zeros((X.shape[0], len(base_models)))

# ✅ K-Fold를 사용하여 Base Models 학습 및 예측
for train_idx, val_idx in kf.split(X, y):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    for i, (name, model) in enumerate(base_models.items()):
        model.fit(X_train_fold, y_train_fold)
        base_model_preds[val_idx, i] = model.predict_proba(X_val_fold)[:, 1]

# ✅ Base Model 예측 확률을 logit 변환 (log-odds 값)
base_model_preds = logit(np.clip(base_model_preds, 1e-6, 1 - 1e-6))  # ✅ logit 변환

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:10:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.089459 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 778
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:12:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.155278 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 773
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054554
[LightGBM] [Info] Start training from score -1.054554


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:15:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.134357 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 773
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054554
[LightGBM] [Info] Start training from score -1.054554


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:17:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152099
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.092422 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 775
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258347 -> initscore=-1.054579
[LightGBM] [Info] Start training from score -1.054579


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:20:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152099
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.133240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 772
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258347 -> initscore=-1.054579
[LightGBM] [Info] Start training from score -1.054579


In [ ]:
# ✅ Meta Model 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 600),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'random_state': 42
    }

    auc_scores = []
    for train_idx, val_idx in kf.split(X, y):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

        model = XGBClassifier(**params, eval_metric="logloss", n_jobs=1)
        model.fit(X_train_fold, y_train_fold)
        y_val_pred = model.predict_proba(X_val_fold)[:, 1]
        auc_scores.append(roc_auc_score(y_val_fold, y_val_pred))

    return np.mean(auc_scores)

# ✅ Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2025-02-12 13:22:24,384] A new study created in memory with name: no-name-65550695-5496-4dbd-960c-9e4e0b3597a3
<ipython-input-14-3af9fec04118>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
[I 2025-02-12 13:24:12,375] Trial 0 finished with value: 0.7290482213036903 and parameters: {'n_estimators': 507, 'learning_rate': 0.005116723788000946, 'max_depth': 3, 'subsample': 0.7294122326241438, 'colsample_bytree': 0.8970313850495156}. Best is trial 0 with value: 0.7290482213036903.
<ipython-input-14-3af9fec04118>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_logunif

In [ ]:
# ✅ 최적의 하이퍼파라미터 적용
best_params = study.best_params
meta_model = XGBClassifier(**best_params, eval_metric="logloss", n_jobs=1)
meta_model.fit(base_model_preds, y)

# ✅ 1단계 Meta Model 예측값 저장
first_meta_preds = meta_model.predict_proba(base_model_preds)[:, 1]

# ✅ 2단계 Stacking Meta Model (LogisticRegression 사용)
X_stack = np.column_stack([base_model_preds, first_meta_preds.reshape(-1, 1)])
second_meta_model = LogisticRegression()
second_meta_model.fit(X_stack, y)

LogisticRegression()

In [ ]:
# ✅ 최종 예측 수행
final_preds = second_meta_model.predict_proba(X_stack)[:, 1]
auc_score = roc_auc_score(y, final_preds)
print(f"✅ Double Stacking ROC-AUC: {auc_score:.4f}")

✅ Double Stacking ROC-AUC: 0.7409


In [ ]:
# ✅ 테스트 데이터 예측
test_base_preds = np.zeros((df_test.shape[0], len(base_models)))
for i, (name, model) in enumerate(base_models.items()):
    test_base_preds[:, i] = model.predict_proba(df_test)[:, 1]

test_base_preds = logit(np.clip(test_base_preds, 1e-6, 1 - 1e-6))
test_first_meta_preds = meta_model.predict_proba(test_base_preds)[:, 1]
X_test_stack = np.column_stack([test_base_preds, test_first_meta_preds.reshape(-1, 1)])
test_preds = second_meta_model.predict_proba(X_test_stack)[:, 1]

In [ ]:
# ✅ 제출 파일 생성
submission = pd.DataFrame({'ID': df_test_ids, 'probability': test_preds})
submission.to_csv('submission_14.csv', index=False)
print("✅ Submission file saved: submission_14.csv")

✅ Submission file saved: submission_14.csv
